In [40]:
# Name: Abdullahi Sani Shuaibu
# ID: 202303370
# Project: Embedded Markov Proces based Model for Performance Analysis of IDPS 
# Course: COE 520 (Queuing Theory and Network Applications)
# Instructor: Dr. Mohammad Abdurazzak Felemban
# Term: 231
# Date: 6 October, 2023

### Constants

In [41]:
L = 3 # L-1 is queue length
p = 0.3 # probability of early decision
r = 1 - p # probability of content checking

### Settings

In [42]:
micro = 10**-6;
settings  = {
    1: {'alpha': (1/0.5)*micro, 'mu': (1/4)*micro},
    2: {'alpha': (1/0.5)*micro, 'mu': (1/8)*micro},
    3: {'alpha': (1/0.5)*micro, 'mu': (1/12)*micro},
    4: {'alpha': (1/5.0)*micro, 'mu': (1/12)*micro},
    }

### Coefficients

In [43]:
w01 = w02 = 1
w11 = lambda setting, lam: ((setting['alpha'] + lam)*lam)/((setting['alpha'] + p)*setting['mu'])
w12 = lambda setting, lam: (lam - p*lam)/(setting['alpha'] + p*lam)

def wn1(n, setting, lam):
    """ for 2 <= n < L """
    if n == 0:
        return w01
    if n == 1:
        return w11(setting, lam)
    return (((setting['alpha'] + lam)*(lam + setting['mu']))/((p*lam + setting['alpha'])*setting['mu']))*wn1(n-1, setting, lam) - ((lam + setting['alpha'])/(lam*p + setting['alpha']))*wn1(n-2, setting, lam) - ((lam*setting['alpha'])/((lam*p + setting['alpha'])*setting['mu']))*wn2(n-1, setting, lam)

def wn2(n, setting, lam):
    """ for 2 <= n < L """
    if n == 0:
        return w02
    if n == 1:
        return w12(setting, lam)
    return (lam/(lam + setting['alpha']))*wn2(n-1, setting, lam) + (((1 - p)*setting['mu'])/(lam + setting['alpha']))*wn1(n, setting, lam)

# print(wn1(15, settings[1], 5))
# print(wn2(15, settings[1], 5))

### Probabilities

In [44]:
q00 = lambda setting, lam: 1/(1 + sum([wn1(n, setting, lam) + wn2(n, setting, lam) for n in range(1, L+1)]))

def qn1(n, setting, lam):
    if n == L and L > 1:
        return ((lam)/(setting['mu']))*wn1(L-1, setting, lam)*q00(setting, lam)
    if n == L and L == 1:
        return ((lam)/(setting['mu']))*q00(setting, lam)
    return wn1(n, setting, lam)*q00(setting, lam)
    
def qn2(n, setting, lam):
    if n == L and L > 1:
        return ((lam)/(setting['alpha']))*(wn2(L-1, setting, lam) + (1-p)*wn1(L-1, setting, lam))*q00(setting, lam)
    if n == L and L == 1:
        return ((1-p)*lam)/(setting['alpha'])*q00(setting, lam)
    return wn2(n, setting, lam)*q00(setting, lam)
    
# print('q00: ', q00(settings[2], 5))
# print('q21: ', qn1(2, settings[2], 5))
# print('q22: ', qn2(2, settings[2], 5))

q00:  8.437567992261056e-26
q21:  7.500006624996918e-09
q22:  1.312500636342969e-16


## Performance Metrices

#### Throughput

In [ ]:
def throughput(setting, lam):
    return p*setting['mu']*sum([qn1(n, setting, lam) for n in range(1, L+1)]) + setting['alpha']*sum([qn2(n, setting, lam) for n in range(1, L+1)])

#### Probability of Packet Loss

In [ ]:
def ppl(setting, lam):
    return qn1(L, setting, lam) + qn2(L, setting, lam)

#### Average Number of Packets in the System

In [ ]:
def avgn(setting, lam):
    return sum([n*(qn1(n, setting, lam) + qn2(n, setting, lam)) for n in range(1, L+1)])

#### Average Packet Time in the System

In [45]:
def avgt(setting, lam):
    return avgn(setting, lam)/throughput(setting, lam)

#### Average Service Time of the Two Stages

In [ ]:
def avgst(setting):
    return (1/setting['mu']) + ((1-p)/setting['alpha'])

#### Average Packet Queue Time

In [ ]:
def avgqt(setting, lam):
    return avgt(setting, lam) - avgst(setting)

## Visualizations for Security Levels (L=25, p=0.3)

### Throughput

### Packet Loss Ratio

### Average Packet Time in System

## Visualizations for System Population (Security Level 1, p=0.3)

### Throughput

### Packet Loss Ratio

### Average Packet Time in System